# Movie Recommendation Model for Netflix Dataset

 Jingyi Wu 99151300

# Introduction


# Exploration

The dataset contains four attributes: movie id, user id, rating, rating date. 

due to the computer cost, only use first 4499 movies' rate for experiement.

# Methodology

# Evaluation



## Off-line Experimental Setting
accuracy
coverage
serendipity
real-time
### Metrics
#####   RMSE - root mean squared error
 

#####   MAE - mean absolute error

# Ethical

In [ ]:
privacy

spread bad information 


# Conclusion

# Rererence



# Appendix

A2_exploration_and_modeling
RBM
MF